In [10]:
import numpy as np
import scipy as sp
import matplotlib.pylab as plt
import pandas as pd
import glob

import seaborn

In [98]:
class SpongeSVD:
    def __init__(self):
        '''
        This method initializes and loads all of the csv data from the data folder. 
        Input: none
        Output: none

        This method stores all of the necessary information as internal attributes of this class
        '''
        allData={}
        # matrix=[]
        for fileID in glob.glob( '*/**.csv' ):
            name=fileID.split('/')[1].split('.')[0]
            allData[name]=pd.read_csv(fileID, header=None).apply(lambda x: [y if y == 0 else 1 for y in x])
            # matrix.append(allData[name].values.flatten())
        
        self.allData=allData
        # self.matrix=np.array(matrix)
    
    def ConvertDataToMatrix(self):
        '''
        This method converts all data to the corresponding matrix format for SVD
        Input: none
        Output: none

        This stores internally the information needed for the SVD
        '''
        matrix=[]
        allData=self.allData
        for name in allData:
            matrix.append(allData[name].values.flatten())
        self.matrix=np.array(matrix)



    def PlotMatrix(self, name):
        '''
        This method is a plotter fucntion for vizializing stored matrices
        Input: name of the matrix you would like to plot
        Output: none

        This method plots the matrix
        '''
        plt.figure()
        Matrix=self.allData[name]
        plt.imshow(Matrix,origin='lower',cmap='binary')
        ax = plt.gca()
        ax = plt.gca()

        # # Major ticks
        ax.set_xticks([])
        ax.set_yticks([])

        # # Labels for major ticks
        # ax.set_xticklabels(np.arange(1, Matrix.shape[1]+1, 1))
        # ax.set_yticklabels(np.arange(1, Matrix.shape[0]+1  , 1))

        # Minor ticks
        ax.set_xticks(np.arange(-.51, Matrix.shape[1], 1), minor=True)
        ax.set_yticks(np.arange(-.51, Matrix.shape[0], 1), minor=True)

        ax.xaxis.set_ticks_position('none') 
        ax.yaxis.set_ticks_position('none') 


        # Gridlines based on minor ticks
        ax.grid(which='minor', color='black')
    
    def GetAllNames(self):
        '''
        This provides all of the names of the stored variables in the matrix
        Input: none
        Output: keys of the names
        '''
        return self.allData.keys()
    
    def DeleteDesign(self,name):
        '''
        This method helps delete desings not to be inlcuded in the analysis.
        Input: name of design in string or list format
        Output: none
        
        This deletes a design from the class
        '''
        if type(name)==str:
            del self.allData[name]
        else:
            for i in name:
                del self.allData[i]


In [101]:
my=SpongeSVD()
my.DeleteDesign(['Cyl','Uni','Bi'])
# for name in my.GetAllNames():
#     my.PlotMatrix(name)

my.ConvertDataToMatrix()
my.matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [ ]:
mehdi was here...jk lol-"matt"